<a href="https://colab.research.google.com/github/0tanh/Draw-Steel-Adventure-Noticeboard/blob/main/basic_database_structure_for_wrapping_supabase_db_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a common and robust architecture. Wrapping your database in a Flask API is a great practice for separating concerns, managing security, and allowing other services (like a web dashboard) to access the same data later.

Here is a breakdown of the basic structure for each component, along with detailed references.

### 1\. Architectural Overview (The Data Flow)

The flow of information will look like this:

1.  **Discord User:** Types a slash command (e.g., `/profile`).
2.  **Discord Bot (`discord.py`):** Receives the command. It makes an **HTTP request** (using `aiohttp`) to your Flask API (e.g., `GET /api/user/123456789...`).
3.  **Flask API (`Flask`):** A web server running separately. It receives the HTTP request, validates it, and then uses the **`supabase-py` library** to query your database.
4.  **Supabase (Database):** Executes the query (e.g., `SELECT * FROM users WHERE discord_id = '123456789...'`) and returns the data to Flask.
5.  **Flask API:** Formats the data as **JSON** and sends it back to the bot as the HTTP response.
6.  **Discord Bot:** Parses the JSON response and formats a user-friendly message to send back to the Discord channel.

-----

### 2\. Component 1: Supabase Database Structure (Example)

In your Supabase project, you'd create your tables. Let's use a simple example: a `users` table to store basic info and a `logs` table to log actions.

You can create these using the Supabase SQL Editor:

```sql
-- Create a table for basic user information
CREATE TABLE users (
  id BIGINT GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
  discord_id BIGINT NOT NULL UNIQUE,
  username TEXT NOT NULL,
  join_date TIMESTAMPTZ DEFAULT NOW()
);

-- Create a table for logging user actions
CREATE TABLE logs (
  id BIGINT GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
  user_discord_id BIGINT NOT NULL,
  action TEXT NOT NULL,
  timestamp TIMESTAMPTZ DEFAULT NOW(),
  
  -- Optional: Set up a foreign key relationship
  -- This links 'user_discord_id' to the 'discord_id' in the 'users' table
  -- Note: We link to discord_id (which is UNIQUE) instead of the primary key 'id'
  -- for easier querying from the bot.
  FOREIGN KEY (user_discord_id) REFERENCES users (discord_id)
);
```

-----

### 3\. Component 2: The Flask API (The Wrapper)

This is a separate Python application. You'll need to install `flask` and `supabase-py`.

`pip install flask python-dotenv supabase-py`

**Project Structure:**

```
/my-api
  ├── .env
  ├── app.py
  └── requirements.txt
```

**.env (Store your secrets here)**
*Never* hardcode your keys in `app.py`.

```ini
# Get these from your Supabase project settings (Project API)
SUPABASE_URL="https://YOUR_PROJECT_ID.supabase.co"
# Use the 'service_role' key for server-side actions. KEEP THIS SECRET.
SUPABASE_KEY="YOUR_SERVICE_ROLE_KEY"
```

**app.py (The Flask Server)**

In [ ]:
import os
from flask import Flask, jsonify, request
from supabase import create_client, Client
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Initialize Flask app
app = Flask(__name__)

# Initialize Supabase client
url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)

# --- User Routes ---

@app.route('/user/<int:discord_id>', methods=['GET'])
def get_user(discord_id):
    """
    Fetches a specific user by their Discord ID.
    """
    try:
        data, count = supabase.table('users').select('*').eq('discord_id', discord_id).execute()

        # data is a list, check if it has content
        if data[1]: # data[1] contains the list of results
            return jsonify(data[1][0])
        else:
            return jsonify({"error": "User not found"}), 404

    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/user', methods=['POST'])
def create_user():
    """
    Creates a new user.
    Expects JSON data: {"discord_id": 123, "username": "TestUser"}
    """
    user_data = request.json

    if not user_data or 'discord_id' not in user_data or 'username' not in user_data:
        return jsonify({"error": "Missing required data"}), 400

    try:
        data, count = supabase.table('users').insert({
            'discord_id': user_data['discord_id'],
            'username': user_data['username']
        }).execute()

        return jsonify(data[1][0]), 201 # 201 = Created

    except Exception as e:
        # Handle potential errors, e.g., if discord_id is not unique
        return jsonify({"error": str(e)}), 500

# --- Log Routes ---

@app.route('/log', methods=['POST'])
def create_log():
    """
    Creates a new log entry.
    Expects JSON data: {"discord_id": 123, "action": "Joined server"}
    """
    log_data = request.json

    if not log_data or 'discord_id' not in log_data or 'action' not in log_data:
        return jsonify({"error": "Missing required data"}), 400

    try:
        data, count = supabase.table('logs').insert({
            'user_discord_id': log_data['discord_id'],
            'action': log_data['action']
        }).execute()

        return jsonify(data[1][0]), 201

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Run the app
if __name__ == '__main__':
    # Run on 0.0.0.0 to be accessible from outside the container (if using Docker)
    # or from other machines on the network.
    app.run(host='0.0.0.0', port=5000, debug=True)

-----

### 4\. Component 3: The Discord Bot (The Client)

This is your main bot script. It **does not** need the `supabase-py` library. It needs `discord.py` and `aiohttp` (which usually comes with `discord.py`).

`pip install discord.py python-dotenv`

**Project Structure:**

```
/my-bot
  ├── .env
  ├── bot.py
  └── requirements.txt
```

**.env**

```ini
DISCORD_TOKEN="YOUR_BOT_TOKEN_FROM_DISCORD_DEV_PORTAL"
# This is the URL where your Flask API is running
API_BASE_URL="http://127.0.0.1:5000"
```

**bot.py (The Discord Bot)**

In [ ]:
import discord
import os
import aiohttp # For async HTTP requests
from discord.ext import commands
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
TOKEN = os.getenv('DISCORD_TOKEN')
API_URL = os.getenv('API_BASE_URL')

# Set up the bot
intents = discord.Intents.default()
intents.message_content = True # Enable message content intent if needed
bot = commands.Bot(command_prefix='!', intents=intents)

# Define a reusable async function to 'register' a user in our DB via the API
async def ensure_user_in_db(user: discord.User):
    """Checks if a user exists in the DB via the API, if not, creates them."""
    async with aiohttp.ClientSession() as session:
        # 1. Try to get the user
        async with session.get(f"{API_URL}/user/{user.id}") as response:
            if response.status == 200:
                # User exists, all good
                return await response.json()

            if response.status == 404:
                # User not found, let's create them
                payload = {
                    "discord_id": user.id,
                    "username": user.name
                }
                async with session.post(f"{API_URL}/user", json=payload) as post_response:
                    if post_response.status == 201:
                        print(f"Successfully created user: {user.name}")
                        return await post_response.json()
                    else:
                        print(f"Error creating user: {await post_response.text()}")
                        return None
            else:
                print(f"Error checking user: {await response.text()}")
                return None

# --- Bot Events ---

@bot.event
async def on_ready():
    print(f'Logged in as {bot.user}!')
    print('------')

@bot.event
async def on_member_join(member):
    """When a new member joins, add them to the database via the API."""
    await ensure_user_in_db(member)

    # Also, log this action via the API
    async with aiohttp.ClientSession() as session:
        payload = {"discord_id": member.id, "action": "Joined server"}
        await session.post(f"{API_URL}/log", json=payload)

    await member.guild.system_channel.send(f"Welcome {member.mention}! I've added you to our records.")

# --- Bot Commands ---

@bot.command()
async def profile(ctx, member: discord.Member = None):
    """Fetches a user's profile from the database via the API."""
    if member is None:
        member = ctx.author

    # Make sure the user is in the DB
    user_data = await ensure_user_in_db(member)

    if user_data:
        # Create a nice embed to show the data
        embed = discord.Embed(
            title=f"{member.name}'s Profile",
            color=discord.Color.blue()
        )
        embed.set_thumbnail(url=member.avatar.url)
        embed.add_field(name="Database ID", value=user_data.get('id', 'N/A'), inline=False)
        embed.add_field(name="Discord ID", value=user_data.get('discord_id', 'N/A'), inline=False)

        # Format the date nicely
        join_date_str = user_data.get('join_date', 'N/A')
        if join_date_str != 'N/A':
            # Simple parsing; for complex ISO strings, use datetime.fromisoformat
            join_date_str = join_date_str.split('T')[0]

        embed.add_field(name="Joined DB", value=join_date_str, inline=False)

        await ctx.send(embed=embed)
    else:
        await ctx.send(f"Could not find or create a profile for {member.name}.")


# Run the bot
bot.run(TOKEN)

-----

### 6\. Detailed References

Here are the official docs you'll need to build on this foundation:

  * **Supabase (Database):**

      * **Main Docs:** [Supabase Official Documentation](https://supabase.com/docs)
      * **Table Editor:** [How to use the Supabase table editor](https://supabase.com/docs/guides/database/tables)
      * **Python Client (`supabase-py`):** [GitHub Repository (with examples)](https://github.com/supabase-community/supabase-py)
      * **Filtering Data:** [supabase-py "Filters"](https://www.google.com/search?q=https://supabase-community.github.io/supabase-py/usage/querying/%23filters) (e.g., `.eq()`, `.gt()`, etc.)

  * **Flask (API):**

      * **Official Tutorial:** [Flask Quickstart](https://flask.palletsprojects.com/en/3.0.x/quickstart/)
      * **JSON Responses:** [Flask `jsonify` function](https://www.google.com/search?q=%5Bhttps://flask.palletsprojects.com/en/3.0.x/api/%23flask.json.jsonify%5D\(https://flask.palletsprojects.com/en/3.0.x/api/%23flask.json.jsonify\))
      * **Receiving Data:** [Flask Request Object](https://www.google.com/search?q=https://flask.palletsprojects.com/en/3.0.x/quickstart/%23accessing-request-data)

  * **Discord.py (Bot):**

      * **Official Docs:** [discord.py Documentation](https://discordpy.readthedocs.io/en/stable/)
      * **Bot Commands:** [Intro to Bot Commands](https://discordpy.readthedocs.io/en/stable/ext/commands/commands.html)
      * **Embeds:** [How to use Embeds](https://www.google.com/search?q=https://discordpy.readthedocs.io/en/stable/guides/embeds.html)

  * **Async HTTP (for Bot):**

      * **`aiohttp` Docs:** [Client Usage (Quickstart)](https://docs.aiohttp.org/en/stable/client_quickstart.html) (Essential for making non-blocking requests from your bot).